Preprocesing Text menggunakan NLTK

Nicholaus Verdhy || A11.2020.12447

# Penjelasan Singkat mengenai NLTK

NLTK adalah kumpulan alat dan program yang digunakan untuk memproses dan menganalisis bahasa alami (NLP) dalam bahasa Inggris. Ini ditulis dalam bahasa pemrograman Python dan digunakan untuk tugas-tugas NLP yang melibatkan analisis teks dan statistik[1]

# Project 1 : Preprocessing dengan NLTK

## Import Library

In [1]:
import re
import string
import time
import pandas as pd
from copy import deepcopy


#Untuk Preprocessing Data nya
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

#import nltk
import nltk
from nltk.tokenize import word_tokenize

#ignore warning
import warnings
warnings.filterwarnings('ignore')

Penjelasan untuk setiap library
- `import re` digunakan untuk bekerja dengan ekspresi reguler (regular expressions), yang memungkinkan Anda untuk melakukan pencarian dan manipulasi teks berdasarkan pola-pola tertentu.digunakan untuk pencarian dan manipulasi teks berdasarkan pola pola tertentu
- `import string` digunakan dalam manipulasi teks, seperti menghapus tanda baca atau mengoperasikan karakter-karakter tertentu 
- `import time` digunakan untuk bekerja dengan waktu dan tanggal dalam Python.
- `import pandas` digunakan untuk memuat dataset yang ada
- `from copy import deepcopy`  ini gunanya, ketika kita melakukan duplikasi dari objek, maka hasil duplikasinya adalah objek baru dan tidak terpengaruhi oleh objek lama
- `from ekphrasis.classes.preprocessor import TextPreProcessor` untuk melakukan berbagai tugas pemrosesan teks, terutama pada teks yang berasal dari media sosial.
- `from ekphrasis.classes.tokenizer import SocialTokenizer`adalah tokenizer yang digunakan untuk membagi teks menjadi kata-kata
- `from ekphrasis.dicts.emoticons import emoticons` adalah kamus yang mungkin digunakan untuk mengganti emotikon dengan kata-kata yang sesuai.
- `import nltk` adalah pernyataan impor untuk mengimpor modul NLTK (Natural Language Toolkit), yang merupakan pustaka yang digunakan dalam pemrosesan bahasa alami (NLP)
- `from nltk.tokenize import word_tokenize` adalah salah satu alat dalam NLTK yang digunakan untuk membagi teks menjadi kata-kata atau token. 
- `import warning` mengabaikan warning

## Loading Dataset

In [2]:
df = pd.read_csv("Dataset_Sentimen_Emosi.csv")

In [3]:
df.head()

,Tweet,Sentimen,Emosi
0,"Cegah mata rantai Covid-19,mari kita dirumah s...",1.0,1
1,aku mohon yaAllah semoga wabah covid-19 menghi...,1.0,-1
2,Pemprov Papua Naikkan Status Jadi Tanggap Daru...,1.0,1
3,Covid belum nyampe prigen mbak hmm hoax,0.0,-2
4,"Nyuruh orang pintar, lu aja Togog. Itu kerumun...",-1.0,-2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Tweet     904 non-null    object 
 1   Sentimen  903 non-null    float64
 2   Emosi     904 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 21.3+ KB


Deskripsi Kolom:

`Tweet` : Isi hasil tweet orang di twiter

`Sentimen`  : mengindikasikan sentimen atau perasaan yang terkait dengan setiap tweet. Sentimen dapat berupa positif, negatif, netral, atau mungkin memiliki label lebih rinci seperti senang, marah, sedih, dan lain-lain.

`Emosi`      : sama seperti sentimen

## Data Cleansing

Melakukan pengecekan missing value pada dataset yang ada

In [7]:
df.isna().sum()

Tweet       0
Sentimen    1
Emosi       0
dtype: int64

Terdapat 1 data missing value pada kolom Sentimen, mari kita hapus dengan dropna

In [8]:
df.dropna(axis=0,inplace=True)

In [9]:
df.isna().sum()

Tweet       0
Sentimen    0
Emosi       0
dtype: int64

Data sudah bersih, kita lanjutkan pada tahap PreProcessingg data

## Preprocessing Data

Sumber modul : https://github.com/cbaziotis/ekphrasis 

Modulnya ekphrasis

In [5]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


PENJELASAN SINGKAT  TEXT_PROCESOR DARI EKPHARASIS

Jadi **`TextPreProcessor`** adalah pustaka dari Ekpharasis. TextPreProcessing ini digunakan untuk memproses teks dalam berbagai cara sesuai dengan konfigurasi yang telah ditentukan. Teks yang diambil biasanya diambil dari media sosial.

1. `normalize` : ini adalah kata kata yang dinormalisasikan dalam teks. Yang nantinya mengubah kata kata/ frasa tertentu menjadi bentuk yang lebih umum. Misal di dalam kode kita ada email, percent, money, phoinoe dan linnya. jika kita memiliki kalimat " Nico mempuinyai email nicho@gmail.com dengan nomor hp 00000" maka akan dinormalisasikan menjadi : `<user>`Nicho`<user>` mempunyai email `<email>`nicho@gmail.com`<email>` dengan nomor hp `<phone>` 00000`<phone>` .

2. `annotate` : ini gunanya untuk mengubah bentuk yang diberi anotasi didalam kata/ kalimat. Anotasinya berupa  hastag (hastag), berupapengulaaaaaan (elonganted), KAPITALSMUA (allcaps), Woo woo woo (repeated), Amazing!!!!(Emphasis), F** and S**t (censored). hasil nya nanti seperti notmalize. Kata tersebut akan dinormalisasikan

3. `fix_html` : fix html ini digunakan perbaiakan token token html dalam teks.

4. `segmenter` dan `corrector` : segementer lebih untuk menyuruh ekpharasis untuk menggunakan korpus bahasa yang biasanya ada di twitter, lalu corrector melakukan koreksi ejaan yang bereferensikan dari twitter.

5. `unpack_hashtags=True` : Dengan pengaturan ini, Ekphrasis akan melakukan pemisahan kata pada hashtag. dari #contoh menjadi 'contoh'

6. `unpack_contractions=True` : Dengan pengaturan ini, Ekphrasis akan melakukan pemisahan pada kontraksi bahasa. Misalnya, kontraksi "can't" akan dipisahkan menjadi "can" dan "not."

7. `spell_correct_elong=False` : Jika diatur sebagai `True` , Ekphrasis akan melakukan koreksi ejaan pada kata-kata yang memiliki karakter pengulangan (seperti "sooo" menjadi "so"). Namun, dalam pengaturan ini (diatur sebagai False), koreksi ejaan untuk kata-kata yang diulang tidak akan dilakukan.

8. `tokenizer`: Ini adalah konfigurasi untuk pemilihan tokenizer yang digunakan dalam pemrosesan teks. Mmenggunakan SocialTokenizer dengan opsi `lowercase=True`. Tokenizer ini akan **memecah teks** menjadi sejumlah token berdasarkan **spasi**, dan semua token akan **dikonversi menjadi huruf kecil (lowercase)**.

9. `dicts=[emoticons]` : Ini adalah konfigurasi yang memungkinkan Anda untuk mengganti token dalam teks dengan ekspresi lain. Dalam pengaturan ini, emoticons digunakan sebagai kamus untuk mengganti emotikon dalam teks dengan ekspresi tertentu.

In [10]:
#1. memanggil ekhparasis yang sudah dibuat
def pembersihan_data(text):
    return " ". join(text_processor.pre_process_doc(text))

#fungsi ams 1-3

#2. mengubah non ascii jadi ascii
def mengganti_non_ascii(text):
    return text.encode('ascii','replace').decode('ascii')

#3. menghapus spasi ekstra dalam teks
def menhapus_extra_spasi(text):
    return " ".join(text.split()) #dikembalikan menjadi single space, lalu dilakukan join pada text, lalu dilakukan split.

#4. Menghapus emoji/ emotikon
def menghapus_emoticon_emoji(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

#5. menghapus karakter tab (\t), baris baru (\n), dan karakter khusus
def hapus_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

#6. Menggantikan beberapa spasi berturut-turut dengan satu spasi tunggal
def ganti_spasi_tunggal(text):
    return re.sub('\s+',' ',text) # jadi jika pada kalimat itu terdapat tab baru, baris baru alan digantikan dengan " "

#7. Menghapus kata RT ( Retweet)
def hapus_ReTweet(text):
    return text.replace('RT'," ") # dari kata RT, di replace jadi kosong " "

#8 Hapus elemen elemen mention 
def hapus_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split()) #dari karaketer yang ada di replace menajadi white sapce, lalau di split

#9 hapus url
def hapus_url(text):
    return text.replace("http://", " ").replace("https://", " ") #akan menggantikan http:// dan htpps:// menjadi whitesapce

#10 hapus single karakter
def mnenghapus_single_karakter(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text) # jika ada karakter huruf yang tunggal atau tidak diapit hutuf lain akan dihapus

#11 hapus titik extra
def menghapus_titik_extra(text):
    return text.replace('..'," ") # menghapus extra ... yang berlebihan menjadi .

#12 mengahapus strip
def mengganti_strip(text):
    return text.replace('-'," ") #ketika ada kalimat mengadung -, maka akan direplace dengan whitespace " "

#13 Membuat smua huruf jadi kecil
def huruf_kecil_semua(text):
    return text.lower() #membuat semua teks menjadi huruf non kapital

#14. Menghapus spasi di awal kalimat dan ahkir kalimat
def hapus_spasi_awal_ahkir_kalimat(text):
    return text.strip() #jadi strip() gunanya untuk menghapus spasi depan dan blakang

#15. hapus Spasi berlebihan
def hapus_spasi_dobel(text):
    return re.sub('\s+',' ',text) #menahpus spasi yang berlebihan

#16. Mengapus tanda baca, kecuali tanda hubung
def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 

#17. hapus untuk <>
def hapus_number_eks(text):
    return text.replace('<number>'," ")
#18. hapus angka
def hapus_angka(text):
    return re.sub(r"\d+", "", text) 
#19. Hapus teks URL
def hapus_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ") #jadi teks URL atau url jadi white sapce
#20. Pengaturan Spasi
def pengaturan_spasi(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text) #memberikan spasi sebelum tanda baca

Penjelasan kode : 

1. `pembersihan_data(text)`: Menggunakan `text_processor` (yang telah diinisialisasi sebelumnya) untuk membersihkan dan memproses teks dokumen.

2. `mengganti_non_ascii(text)`: Menggantikan karakter non-ASCII dengan karakter yang sesuai dalam ASCII. Karakter non-ASCII akan digantikan dengan tanda tanya atau karakter lainnya sesuai aturan penggantian.

3. `menhapus_extra_spasi(text)`: Menghapus spasi ekstra dalam teks, menggantinya dengan satu spasi tunggal.

4. `menghapus_emoticon_emoji(text)`: Menghapus emotikon atau emoji dalam teks dengan menggantikannya dengan spasi.

5. `hapus_tab(text)`: Menghapus karakter tab (`\t`), baris baru (`\n`), dan karakter khusus lainnya dari teks.

6. `ganti_spasi_tunggal(text)`: Menggantikan beberapa spasi berturut-turut dengan satu spasi tunggal.

7. `hapus_ReTweet(text)`: Menghapus kata "RT" (Retweet) dari teks.

8. `hapus_mention(text)`: Menghapus elemen-elemen mention atau tag dalam teks, seperti "@username".

9. `hapus_url(text)`: Menghapus URL dari teks, baik yang dimulai dengan "http://" maupun "https://".

10. `mnenghapus_single_karakter(text)`: Menghapus karakter tunggal yang tidak diapit oleh huruf lain.

11. `menghapus_titik_extra(text)`: Menghapus tanda titik yang berlebihan dan menggantikannya dengan spasi.

12. `mengganti_strip(text)`: Menggantikan tanda strip (-) dengan spasi.

13. `huruf_kecil_semua(text)`: Mengubah semua huruf dalam teks menjadi huruf kecil.

14. `hapus_spasi_awal_ahkir_kalimat(text)`: Menghapus spasi di awal dan akhir kalimat.

15. `hapus_spasi_dobel(text)`: Menghapus spasi berlebihan dalam teks.

16. `remove_punctuation(text)`: Menghapus tanda baca, kecuali tanda hubung (-).

17. `hapus_number_eks(text)`: Menghapus teks "<number>".

18. `hapus_angka(text)`: Menghapus semua angka dalam teks.

19. `hapus_URL_eks(text)`: Menghapus teks "URL" atau "url".

20. `pengaturan_spasi(text)`: Menambahkan spasi sebelum tanda baca, menjaga spasi di antara kata-kata dan tanda baca.


# Referensi

[1]	E. J. Rifano, A. C. Fauzan, A. Makhi, E. Nadya, Z. Nasikin, and F. N. Putra, “Text Summarization Menggunakan Library Natural Language Toolkit (NLTK) Berbasis Pemrograman Python,” Ilk. J. Comput. Sci. Appl. Inform., vol. 2, no. 1, Art. no. 1, Apr. 2020, doi: 10.28926/ilkomnika.v2i1.32.

